# Breast Cancer Prediction with Machine Learning


### 1. Introducció 

El càncer més comú entre les dones és el de mama. L'objectiu d'aquesta pràctica, és predir si un tumor és beginge o maligne. Per fer-ho disposem d'una base de diferents mostres. Per calcular les característiques de cada mostra, es fa a partir d’una imatge digitalitzada d’un aspirat d’agulla fina (FNA) d’una massa mamària. Descriuen les característiques dels nuclis cel·lulars presents a la imatge. D'entre aquestes característiques, tenim les següents:
- ID number
- diagnosis (M si és maligne i B si és benigne)
- radius: mitjana de les distàncies des del centre fins a al punt del perímetre
- texture: desviació estàndard dels valors a escala de grisos
- perimeter: perímetre
- area: àrea
- smothness: variació local de longituds del radi
- compactness: perímetre ^ 2 / àrea - 1,0
- concativy: gravetat de les porcions còncaves del contorn
- concave points: nombre de porcions còncaves del contorn
- simmetry: simetra
- dimensió fractal: aproximació del contorn

La mitjana, error estàndard i "pitjor" d'aquestes funcions es van calcular per a cada imatge, donant lloc a 30 funcions.

### 2. Com són les dades?

#### Importem les llibreries i carreguem  les dades

In [ ]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib notebook
import scipy.stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.linear_model import LogisticRegression

In [ ]:
# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# Carreguem el dataset
dataset = pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")

print("La dimensionalitat de la nostre BBDD (mostres, característiques):", dataset.shape)

In [ ]:
print("Per visualitzar les primeres 5 mostres de la BBDD:")
dataset.head()

In [ ]:
print("Mirem els atribut amb valors no existents:")
print(dataset.isnull().sum())

Tots els atributs són numèrics menys "diagnosis" que és categòric, concretament està en format string. Convertim M en un 1 i B en un 0

In [ ]:
dataset['diagnosis'] = dataset['diagnosis'].map({'M':1 ,'B':0}) 
dataset['diagnosis'].astype('int')

In [ ]:
print("El nostre atribut objectiu serà 'diagnostic', per tant, ens interesa que les proporció de malignes i benignes sigui semblant:")
sns.countplot(dataset['diagnosis'])
dataset['diagnosis'].value_counts()

Un 62% de les dades són de tumors benignes, mentre un 38% són de malignes.

In [ ]:
# Mirem la correlació entre els atributs d'entrada
correlacio = dataset.iloc[:,1:32].corr()
pyplot.figure(figsize=(20,20))
ax = sns.heatmap(correlacio, annot=True, linewidths=.5, fmt='.0%')

### 3. Preparació de dades

Per fer el model, escollirem els atributs amb una correlació major de 0.7:

In [ ]:
(correlacio['diagnosis'])[correlacio['diagnosis']>0.7]

In [ ]:
#Per l'atribut y tindrem l'objectiu
#Per l'atribut x escollim els atributs amb major correlació
data = dataset.values
y = data[:, 1]
x = data[:, [2, 4, 5, 9, 22, 24, 25, 29 ]] 
#x[radius_mean, perimeter_mean, area_mean, concave points_mean, radius_worst, perimeter_worst, area_worst, concave points_worst]es. Para ello, se dispone de p características (X1, X2 … Xp

#### Primera prova amb els models

Separem les dades que tenim en un 80% validació i un 20% test

In [ ]:
x_t, x_v, y_t, y_v = train_test_split(x, y, train_size=0.8)

In [ ]:
#Calculem una regressió logística:
logireg = LogisticRegression(C=2.0, fit_intercept=True, penalty='l2', tol=0.001)
logireg.fit(x_t, y_t)
probs = logireg.predict_proba(x_v)
print ("Correct classification Logistic ", logireg.score(x_v, y_v))

In [ ]:
#Calculem una svm:
svc = svm.SVC(C=10.0, kernel='rbf', gamma=0.9, probability=True)
svc.fit(x_t, y_t)
probs = svc.predict_proba(x_v)
print ("Correct classification SVM  ", svc.score(x_v, y_v))

En la primera prova, al regressió logística és millor que SVM

#### Normalització

In [ ]:
#Podem provar d'estandaritzar algunes dades per comorobar si funciona millor.
def standarize(x):
    mean = x.mean(0)
    std = x.std(0)
    x_t = x - np.array(mean)
    x_t /= np.array(std)
    return x_t

x_s=standarize(x)
#Per tant en x tindrem les dades sense modificar i en x_s les dades normalitzades.

Separem les dades que tenim en un 80% validació i un 20% test

In [ ]:
x_t, x_v, y_t, y_v = train_test_split(x_s, y, train_size=0.8)

In [ ]:
#Calculem una regressió logística:
logireg = LogisticRegression(C=2.0, fit_intercept=True, penalty='l2', tol=0.001)
logireg.fit(x_t, y_t)
probs = logireg.predict_proba(x_v)
print ("Correct classification Logistic ", logireg.score(x_v, y_v))

In [ ]:
#Calculem una svm:
svc = svm.SVC(C=10.0, kernel='rbf', gamma=0.9, probability=True)
svc.fit(x_t, y_t)
probs = svc.predict_proba(x_v)
print ("Correct classification SVM  ", svc.score(x_v, y_v))

Els dos classificadors han millorat en comparació a l'anterior. Per aquest motiu, utilitzarem les dades estandaritzades per les proves següents. De nou, el regressor logístic té un millor accuracy.

#### PCA

Probarem de fer un PCA per reduir-ho a dos dimensions.

In [ ]:
pca = PCA(n_components=2)
x2=x_s
elem = pca.fit_transform(x2)
df=pd.DataFrame(data=elem, columns=['1','2'])
final_df=pd.concat([df, dataset['diagnosis']], axis=1)

In [ ]:
fig = pyplot.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = [0, 1]
colors = ['orange', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = final_df['diagnosis'] == target
    ax.scatter(final_df.loc[indicesToKeep, '1']
               , final_df.loc[indicesToKeep, '2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

Podem obervar que les dades es veuen bastant separades.

#### Transformació polinomial

Fem una transformació polinomial de grau 3:

In [ ]:
# perform a polynomial features transform of the dataset
trans = PolynomialFeatures(degree=3)
X_polynomial = trans.fit_transform(x_s)

In [ ]:
x_t, x_v, y_t, y_v = train_test_split(X_polynomial, y, train_size=0.8)

In [ ]:
logireg = LogisticRegression(C=2.0, fit_intercept=True, penalty='l2', tol=0.001)
logireg.fit(x_t, y_t)
probs = logireg.predict_proba(x_v)
print ("Correct classification Logistic ", logireg.score(x_v, y_v))

In [ ]:
svc = svm.SVC(C=10.0, kernel='rbf', gamma=0.9, probability=True)
svc.fit(x_t, y_t)
probs = svc.predict_proba(x_v)
print ("Correct classification SVM  ", svc.score(x_v, y_v))

Cap dels dos classificadors a millorat, per tant, la transformació polinomial no ñes acertada per les nostres dades.

### 4. Crossvalidation

Utilitzaem la crossvalidació per avaluar els resultats dels models. Per aquest apartat, utilitzarem tres models diferents: regressió logística, SVM i el KNN. Ho farem per k igual a 3, 5 i 10.

In [ ]:
for k in [3,5,10]:
    cv = KFold(n_splits=k, shuffle=True) 
    logireg = LogisticRegression(C=2.0, fit_intercept=True, penalty='l2', tol=0.001)
    svc = svm.SVC(C=10.0, kernel='rbf', gamma=0.9)
    classifier= KNeighborsClassifier()
    # evaluate model
    scores = cross_val_score(logireg, x_s, y, scoring='accuracy', cv=cv, n_jobs=-1)
    scores2 = cross_val_score(svc, x_s, y, scoring='accuracy', cv=cv, n_jobs=-1)
    scores3 = cross_val_score(classifier, x_s, y, scoring='accuracy', cv=cv, n_jobs=-1)
    # report performance
    print('K =',k)
    print('Regressió Logística --> Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
    print('SVM --> Accuracy: %.3f (%.3f)' % (np.mean(scores2), np.std(scores2)))
    print('KNN --> Accuracy: %.3f (%.3f)' % (np.mean(scores3), np.std(scores3)))
cv = LeaveOneOut()
logireg = LogisticRegression(C=2.0, fit_intercept=True, penalty='l2', tol=0.001)
svc = svm.SVC(C=10.0, kernel='rbf', gamma=0.9)
classifier= KNeighborsClassifier()
# evaluate model
scores = cross_val_score(logireg, x_s, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores2 = cross_val_score(svc, x_s, y, scoring='accuracy', cv=cv, n_jobs=-1)
scores3 = cross_val_score(classifier, x_s, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report performance
print('LeaveOneOut')
print('Regressió Logística --> Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
print('SVM --> Accuracy: %.3f (%.3f)' % (np.mean(scores2), np.std(scores2)))
print('KNN --> Accuracy: %.3f (%.3f)' % (np.mean(scores3), np.std(scores3)))

Veiem que per k=10 tenim els millors resultats per la regressió logística i SVM. Pel cas del KNN no canvia.  Un cop hem fet Leave One Out, només milloren els resultats de KNN.

### 5. Mètriques de classificació

En aquest apartat ens centrarem en les mètriques de classificació. Les que utilitzarem seran  `accuracy_score`, `f1_score` i `average_precision_score`

In [ ]:
for acc in ['accuracy','f1','average_precision']:
    cv = KFold(n_splits=5, shuffle=True) 
    logireg = LogisticRegression(C=2.0, fit_intercept=True, penalty='l2', tol=0.001)
    svc = svm.SVC(C=10.0, kernel='rbf', gamma=0.9)
    classifier= KNeighborsClassifier()
    # evaluate model
    scores = cross_val_score(logireg, x_s, y, scoring=acc, cv=cv, n_jobs=-1)
    scores2 = cross_val_score(svc, x_s, y, scoring=acc, cv=cv, n_jobs=-1)
    scores3 = cross_val_score(classifier, x_s, y, scoring=acc, cv=cv, n_jobs=-1)
    # report performance
    print(acc)
    print('Regressió Logística --> Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))
    print('SVM --> Accuracy: %.3f (%.3f)' % (np.mean(scores2), np.std(scores2)))
    print('KNN --> Accuracy: %.3f (%.3f)' % (np.mean(scores3), np.std(scores3)))
    print('\n')

Podem observar que la millor mètrica per accuracy i f1 és la regressió logística mentre que per average_precision és KNN

Ara veurem la classificació de scikit-learn:

In [ ]:
for idx_train, idx_test in cv.split(x_s):
    x_t, y_t = x_s[idx_train], y[idx_train]
    x_test, y_test = x_s[idx_test], y[idx_test]
logireg = LogisticRegression(C=2.0, fit_intercept=True, penalty='l2', tol=0.001)
logireg.fit(x_t, y_t)
y_pred_log = logireg.predict(x_s)
y_true_log = y

In [ ]:
from sklearn.metrics import classification_report
target_names = ['Benigne', 'Maligne']
print(classification_report(y_true_log, y_pred_log, target_names=target_names))

Veiem els seus significats:
- precision seran els elements que es classifiquen correctament entre aquesta classe.
- recall ens proporciona intuitivament la capacitat del classificador per trobar les mostres de la classe.
- f1-score és la mitjana harmònica entre precisió i record.
- support és el nombre d’ocurrències de la classe donada al nostre conjunt de dades.

Podem observar que els tumors benignes tenen millors resultats de predicció que els malignes.

### 6. Resultats

Mostrem la matriu de confussió i el gràfic ROC pel model logístic ja que ha resultat ser el que millor s'adapta a la base de dades:

In [ ]:
cm_log = confusion_matrix(y_true_log,y_pred_log)    
pyplot.subplots(figsize=(10, 6))
cm_log_m = pd.DataFrame(cm_log, index = ['B','M'], columns = ['B','M'])
sns.heatmap(cm_log_m, annot = True, fmt = 'd', cbar=False)
pyplot.xlabel("Predicted")
pyplot.ylabel("Actual")
pyplot.title("Confusion Matrix Logistic Regresion")
pyplot.show()

Els valors de la diagonal són els casos estimats de forma correcte pel model. L'altre diagonal són els valors dels casos que s'ha equivocat:
- 13 dels valors que són malignes, els ha predit com a benignes
- 7 dels valors que són benignes, els ha predit com a malignes

In [ ]:
# roc curve and auc
model = logireg
# generate a no skill prediction (majority class)
ns_probs = [0 for i in range(len(y_t))]
# predict probabilities
lr_probs = model.predict_proba(x_t)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(y_t, ns_probs)
lr_auc = roc_auc_score(y_t, lr_probs)
# summarize scores
print('No Skill: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(y_t, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_t, lr_probs)
# plot the roc curve for the model
pyplot.figure(figsize=(7,5))
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

Les corbes ROC ens proporcionan informació de quan de bo és el nostre predictor. El model ideal seria trobar un AUC d'1. Qualsevol model que es trobés per sota de la lína d'AUC 0.5 no seria eficient. La nostre corba ROC és molt elevada i la probabilitat de que el nostre model faci una predicció correcte és del 98.8%, per tant podem concloure que el nostre model és eficient.

### 7. Conclusions 

- El classificador que millor s'adapta a les nostres dades és el Regressor Logístic.
- Molts dels atributs que se'ns proporcionen son irrelevants, ja que la correlació amb la variable objectiu és molt baixa i tenirlos en compte podria empitjorar el nostre model.
- A partir de les proves que hem fet, hem pogut veure que els tumors benignes es prediuen millor que els malignes, això pot ser degut a que les proporcions de l'atribut objectius són desiguals. 
- També podem veure, a partir de la matriu de confusió, que hi ha més tumors malignes predits com a benignes tal pel motiu que hem comentat anteriorment.

----------- Natalia López 